In [ ]:
import polars as pl
import plotly.io as pio
import plotly.express as px
import plot_theme as pt

In [ ]:
pio.templates.default = "plotly_white+cc"

# Plot top results for each model and llm

In [ ]:
# top_runs = pl.read_parquet("../classifier_grid_search/top_classifier_runs_per_type.parquet")
top_runs = pl.concat([
    pl.read_parquet("../classifier_grid_search/granite_top_results_per_model.parquet"),
    pl.read_parquet("../classifier_grid_search/tulu_top_results_per_model.parquet")
])

In [ ]:
model_replacements = {
    "rf": "Random forest",
    "svc": "Support vector machine",
    "gnb": "Gaussian Naive Bayes",
    "lr": "Logistic regression",
    "knn": "K-nearest neighbors"
}

llm_name = {
    "granite": "Granite-3.1-8b",
    "tulu": "Tulu-3",
}

top_runs = top_runs.with_columns(
    pl.col("model").replace(model_replacements),
    pl.col("llm").replace(llm_name),
)

In [ ]:
model_order = (
    top_runs.group_by("model")
    .agg(pl.mean("mean_test_score"))
    .sort("mean_test_score")["model"]
    .unique(maintain_order=True)
)

In [ ]:
fig = px.violin(
    top_runs,
    x="model",
    y="f1",
    color="llm",
    box=True,
    category_orders={"model": model_order},
    labels={"f1": "F1 score (weighted)", "model": "Classifier type", "llm": "LLM"},
    points=False,
    violinmode="overlay",
)
fig = fig.update_layout(
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
).update_traces(hoveron='kde')
pt.save(fig, "f1_scores_per_classifier")

# Compare embeddings with granite svc

In [ ]:
cmp_df = pl.read_parquet("../classifier_grid_search/granite_top_model_results_per_embedding.parquet")

In [ ]:
embedding_map = {
    "snowflake_embedding_clean": "Snowflake",
    "gte_embedding_clean": "GTE",
}
cmp_df = cmp_df.with_columns(
    pl.col("embedding").map_elements(embedding_map.get, return_dtype=pl.String),
)

In [ ]:
fig = px.violin(
    cmp_df,
    x="embedding",
    y="f1",
    box=True,
    color="embedding",
    category_orders={"embedding": ["Snowflake", "GTE"]},
    points="all",
    labels={"f1": "F1 score (weighted)", "embedding": "Embedding model"},
)
fig = fig.update_layout(showlegend=False).update_traces(hoveron="points+kde")
pt.save(fig, "classifier_embedding_comparison")